In [1]:
import basedosdados as bd
import pandas as pd

## Seleção do período:
- desde 2022-01-01, início do atual mandato da câmara de deputados
- até 2024-04-01, limite do dataset com acesso gratuito 

# Observações

### Passo 1: Análise de texto de leis propostas

Cada linha da base de dados corresponde a uma proposição, que pode ser de variados tipos.
Os que parecem ser mais relevantes são "Projeto de Lei" ou "Redação Final"
A escolha depende de (1) quanto detalhe se quer ter sobre o texto da lei e (2) o quão próximo o texto é da lei que foi votada e talvez aprovada.
- (1) Se apenas a emenda (descrição da proposição) for suficiente, dá pra usar no bag of words a coluna "ementa" das proposições tipo "Projeto de Lei". Se quiser mais detalhe, é melhor usar o texto completo da lei, coluna "teor_completo", das proposições tipo "Redação Final".
- (2) Se quiser o projeto de lei da forma que foi proposto pelo parlamentar, é melhor usar o tipo "Projeto de Lei", se quiser a lei como ela foi talvez aprovada ou votada, é melhor usar "Redação Final"

Caso a escolha seja usar o texto completo ("teor_completo") da "Redação Final", é preciso encontrar um library que faça a leitura do PDF, e resolver o erro que tive. Também vai ter uma lmiitação de performance por armazenar o texto da web na memória.

### Passo 2: Correlação com temas das leis propostas e sua aprovação

Essa base de dados tem outras tabelas relacionadas à votação, também com o ID da proposição. Mas precisa investigar mais pra entender qual proposição representa a votação do PL antes de passar à sanção do presidente da república (último passo).



## Seleção do tipo de proposição: 'Projeto de Lei' ou 'Redação Final' 

In [2]:
# SQL query

list_types = '''
SELECT DISTINCT
  sigla,
  tipo,
FROM `basedosdados.br_camara_dados_abertos.proposicao_microdados` AS dados
WHERE (dados.data > '2022-01-01' AND dados.data < '2024-04-01')
'''

# Read query via 'Base dos dados' Google Cloud API
projects_list = bd.read_sql(list_types, billing_project_id='anpec-base-dos-dados')

print('**EXEMPLO DOS DIVERSOS TIPOS DE PROPOSIÇÃO (TOTAL: 235)**')
for n in projects_list['tipo'][:10]:
    print(n)

Downloading: 100%|████████████████████████████████████████████████████████████████| 235/235 [00:00<00:00, 619.27rows/s]

**EXEMPLO DOS DIVERSOS TIPOS DE PROPOSIÇÃO (TOTAL: 235)**
Emenda/Substitutivo do Senado
Destaque
Proposta
Reclamação
Redação Final
Parecer Vencedor
Indicação à Mesa
Relatório Preliminar
Relatório de Receita
Requerimento de Moção


- [Projeto de Lei](https://www.congressonacional.leg.br/legislacao-e-publicacoes/glossario-legislativo/-/legislativo/termo/projeto_de_lei_pl) - Proposição de uma lei ordinária, vai ser sancionado (aprovado) pelo presidente da república
- [Projeto de Lei Complementar](https://www.congressonacional.leg.br/legislacao-e-publicacoes/glossario-legislativo/-/legislativo/termo/lei_complementar_lc) - adiciona detalhes a uma lei já aprovada
- [Redação Final](https://www.congressonacional.leg.br/legislacao-e-publicacoes/glossario-legislativo/-/legislativo/termo/redacao_final) - texto do projeto de lei a ser votado pelo plenário da câmara

  **Projeto de lei é a lei quando recém proposta, ainda precisando ser votada pela plenária da câmara e senado (quando se aplica). Redação final é o projeto já ajustado pela Câmara e Senado, antes de ir para sanção (aprovação) do Presidente da República**

[Processo legislativo](https://www.camara.leg.br/entenda-o-processo-legislativo/)

## Detalhamento do tipo 'Projeto de Lei'

In [3]:
# SQL query

query_projects = '''
SELECT DISTINCT
  sigla,
  tipo,
  ementa,
  ementa_detalhada,
  palavra_chave,
  situacao_ultimo_status,
  apreciacao_ultimo_status,
  url_teor_proposicao
FROM `basedosdados.br_camara_dados_abertos.proposicao_microdados` AS dados
WHERE (dados.data > '2022-01-01' AND dados.data < '2024-04-01')
AND dados.tipo = 'Projeto de Lei'
'''

# Read query via 'Base dos dados' Google Cloud API
projects = bd.read_sql(query_projects, billing_project_id='anpec-base-dos-dados')

Downloading: 100%|█████████████████████████████████████████████████████████████| 8904/8904 [00:06<00:00, 1461.84rows/s]


In [4]:
# contagem de projetos de lei no período

projects[projects['tipo'] == "Projeto de Lei"]['tipo'].value_counts()

Projeto de Lei    8904
Name: tipo, dtype: int64

In [5]:
# campo de emenda é a descrição do projeto de lei

for n in projects['ementa'][:5]:
    print(n)

Altera a redação da Lei nº 10.779, de 25 de novembro de 2003, que dispõe sobre a concessão do benefício de seguro desemprego, durante o período de defeso, ao pescador profissional que exerce a atividade pesqueira de forma artesanal, para estabelecer prazo para análise de documentos e pagamento do benefício aos pescadores profissionais artesanais.
Denomina "Rei Pelé" o túnel Santos-Guarujá.
Dispõe sobre reconhecimento e transferência de propriedade, importação e registro veículos automotores com mais de trinta anos de fabricação e altera a Lei nº 6.015, de 1973, a Lei nº 8.723, de 1993, a Lei nº 9.503, de 1997, Código de Trânsito Brasileiro, e a Lei nº 14.382, de 2022.
Institui o Programa Nacional de Doação de Sangue, Plaquetas e Medula óssea.
Altera a Lei n° 11.350, de 5 de outubro de 2006, para dispor sobre o adicional de insalubridade aos Agentes Comunitários de Saúde e aos Agentes de Combate às Endemias, na forma do § 10 do art. 198 da Constituição Federal.


In [6]:
# a própria base de dados já tem um label com palavras chaves do projeto de lei

for n in projects['palavra_chave'][:5]:
    print(n)

Alteração, Lei do Seguro-Defeso (2003), concessão, Benefício financeiro, Seguro-desemprego, Defeso, Atividade pesqueira, Pescador, Pescador profissional, criação, Prazo, análise, Documentação, Pagamento.
Denominação, Túnel, Trecho rodoviário, Santos (SP), Guarujá (SP), São Paulo (Estado), jogador de futebol, homenagem póstuma, logradouro público.
Alteração, Código de Trânsito Brasileiro (1997), Lei dos Registros Públicos (1973), Sistema Eletrônico dos Registro Públicos (SERP), Veículo automotor, emissão, Poluente, reconhecimento extrajudicial, transferência, registro, Proprietário, Usucapião, Importação, Veículo de coleção.
Criação, Programa Nacional de doação de sangue, plaquetas e medula óssea, ampliação, Hemocentro, Hemonúcleo, Posto de coleta, Município, campanha educativa. _ Criação, Carteira Nacional Digital do Doador de Sangue, Aplicativo móvel, histórico, doação.
Alteração, Lei Federal, garantia, Agente comunitário de saúde, Agente de combate às endemias, Adicional de insalubri

In [7]:
# status da proposta, não consegui encontrar algo que indique se o projeto de lei foi aprovado

projects['situacao_ultimo_status'].value_counts().head(5)

Tramitando em Conjunto              4343
Aguardando Designação de Relator    1402
Aguardando Parecer                  1231
Pronta para Pauta                    435
Arquivada                            240
Name: situacao_ultimo_status, dtype: int64

In [8]:
# exemplo de ementa

projects['ementa'][1]

'Denomina "Rei Pelé" o túnel Santos-Guarujá.'

In [9]:
# exemplo de texto completo do projeto de lei

project_url = projects['url_teor_proposicao'][1]
project_url

'https://www.camara.leg.br/proposicoesWeb/prop_mostrarintegra?codteor=2397373'

## Detalhamento do tipo 'Redação Final'

In [10]:
# SQL query

query_writing = '''
SELECT DISTINCT
  sigla,
  tipo,
  ementa,
  ementa_detalhada,
  palavra_chave,
  situacao_ultimo_status,
  apreciacao_ultimo_status,
  url_teor_proposicao
FROM `basedosdados.br_camara_dados_abertos.proposicao_microdados` AS dados
WHERE (dados.data > '2022-01-01' AND dados.data < '2024-04-01')
AND dados.tipo = 'Redação Final'
'''

# Read query via 'Base dos dados' Google Cloud API
writing = bd.read_sql(query_writing, billing_project_id='anpec-base-dos-dados')

Downloading: 100%|█████████████████████████████████████████████████████████████| 1559/1559 [00:00<00:00, 2557.16rows/s]


In [11]:
# contagem de redações finais, menos do que projetos de lei, já que ela seria um estágio mais avançad

writing[writing['tipo'] == "Redação Final"]['tipo'].value_counts()

Redação Final    1559
Name: tipo, dtype: int64

In [12]:
# ementa da redação final não é uma descrição tão boa, em muitas linhas só é a referência a um projeto de lei

for n in writing['ementa'][92:97]:
    print(n)

Redação Final do PL 4519/2019
Redação Final do PDL 303/23
Altera a Consolidação das Leis do Trabalho (CLT), aprovada pelo Decreto-Lei nº 5.452, de 1º de maio de 1943, para determinar que as empresas disponibilizem a seus empregados informações sobre campanhas oficiais de vacinação, sobre o papilomavírus humano (HPV) e sobre os cânceres de mama, de colo do útero e de próstata.
Dispõe sobre o direito a indenização por dano moral e a concessão de pensão especial à pessoa com deficiência permanente decorrente de síndrome congênita associada à infecção pelo vírus Zika ou de Síndrome de Guillain-Barré causada pelo vírus Zika, e altera a Consolidação das Leis do Trabalho, aprovada pelo Decreto-Lei nº 5.452, de 1º de maio de 1943, e as Leis nºs 8.742, de 7 de dezembro de 1993, e 8.213, de 24 de julho de 1991.
Redação Final do PDL 284/22


In [13]:
# status da proposta, não consegui encontrar algo que indique se o projeto de lei foi aprovado

writing['situacao_ultimo_status'].value_counts()

Aguardando Providências Internas             400
Pronta para Pauta                             77
Aguardando Apreciação pelo Senado Federal      1
Aguardando Recebimento                         1
Aguardando Leitura e Publicação                1
Name: situacao_ultimo_status, dtype: int64

In [14]:
# Exemplo de ementa

writing['ementa'][1]

'Inscreve o nome de Zilda Arns Neumannno Livro dos Heróis e Heroínas daPátria.'

## Teste de extração da arquivo de PDF da Emenda Final

In [20]:
# Exemplo de texto da redação final

writing_url = writing['url_teor_proposicao'][2]
writing_url

'https://www.camara.leg.br/proposicoesWeb/prop_mostrarintegra?codteor=2201291'

In [16]:
import io
import requests
import PyPDF2

In [31]:
r = requests.get(writing_url)
r.status_code

403

In [39]:
# request forbidden

r = requests.get(writing_url)
r.content

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="pt-br" xml:lang="pt-br">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1" />\n<title>Erro de acesso</title>\n<link rev="made" href="mailto:seseg@camara.leg.br" />\n<link href="https://www.camara.leg.br/proxy-error/pageError.css" rel="stylesheet" type="text/css" />\n</head>\n<body>\n\t<div id="geralError">\n\t\t<div id="barraCamara"></div>\n\t\t<div class="contentError">\n\t\t\t<div class="atencao">\n\t\t\t\t<h1>Desculpe-nos!</h1>\n\t\t\t</div>\n\t\t\t<div class="conteudo">\n\n\t<div class="conteudoBox">\n\t\t<p>Nossos sistemas autom&aacute;ticos de seguran&ccedil;a impediram que a opera&ccedil;&atilde;o fosse conclu&iacute;da.<p>\n\t\t<p>Isso pode ocorrer quando voc&ecirc; copia dados de outros sites e eles v&ecirc;m com conte&uacute;dos n&atilde;o permitidos.<p>\n\t\t<p>Voc&ecirc; p

In [40]:
'''
r = requests.get(writing_url)
f = io.BytesIO(r.content)

reader = PyPDF2.PdfReader(f)
pages = reader.pages
text = "".join([page.extract_text() for page in pages])
'''

'\nr = requests.get(writing_url)\nf = io.BytesIO(r.content)\n\nreader = PyPDF2.PdfReader(f)\npages = reader.pages\ntext = "".join([page.extract_text() for page in pages])\n'